# MongoDB + Pydantic Experiment for RadLex Ontology Data Transformations
* Date: 4/3/2024
* Notebook completed by Adam Lang
* We will do the following in this notebook:
1. Query MongoDB instance.
2. Extract a collection from a database.
3. Transform the text in the collection.
4. Query the collection in the MongoDB for specific unique values.
5. Write the transformed text back to a new collection in MongoDB.

In [6]:
# import local modules
from openimagingdatamodel.ontology_tools import Transform


# import MongoDB modules
import motor.motor_asyncio
from dotenv import dotenv_values

In [7]:
# establish connection to MongoDB
config = dotenv_values(".env")
client = motor.motor_asyncio.AsyncIOMotorClient(config["ATLAS_DSN"])
db = client["ontologies"]
collection = db["RadLex"]
# Get the count of documents in the collection to confirm that the data was loaded
count = await collection.count_documents({})
print(f"Count of documents in the collection: {count}")

Count of documents in the collection: 46761


In [8]:
# Access the ontologies database and "RadLex" collection
doc = await collection.find_one()

In [9]:
print(doc)

{'_id': ObjectId('65f84ed1f80fad5323c79c1f'), 'Class ID': 'http://radlex.org/RID/RID35591', 'Preferred Label': 'string-of-pearls sign of bowel', 'Definitions': 'Oblique or horizontal row of air bubbles visible on abdominal radiograph; almost always indicates small bowel obstruction; air is trapped between valvulae conniventes along the superior wall of the intestine.', 'Obsolete': False, 'Parents': 'http://radlex.org/RID/RID29023', 'http://data': {'bioontology': {'org/metadata/prefixIRI': 'RID35591'}}, 'http://radlex': {'org/RID/Anatomical_Site': 'http://radlex.org/RID/RID132', 'org/RID/Comment': 'http://radiology.rsna.org/cgi/content/full/214/1/157', 'org/RID/Definition': 'Oblique or horizontal row of air bubbles visible on abdominal radiograph; almost always indicates small bowel obstruction; air is trapped between valvulae conniventes along the superior wall of the intestine.', 'org/RID/May_Be_Caused_By': 'http://radlex.org/RID/RID4962', 'org/RID/Preferred_name': 'string-of-pearls s

### MongoDB Query to extract fields in the http://radlex Object field
* We need to query this so we can determine if we have all of the keys in the RadLexProperties pydantic class in the radlex_importer.py file

In [18]:
# Run the MongoDB aggregation query
pipeline = [
    {
        "$addFields": {
            "radlexArray": { "$objectToArray": "$http://radlex" }
        }
    },
    {
        "$unwind": "$radlexArray"
    },
    {
        "$addFields": {
            "radlexField": {
                "$arrayElemAt": [
                    { "$split": [ "$radlexArray.k", "/" ] },
                    -1
                ]
            }
        }
    },
    {
        "$group": {
            "_id": None,
            "uniqueFields": { "$addToSet": "$radlexField" }
        }
    }
]

cursor = collection.aggregate(pipeline)
result = await cursor.to_list(length=110)  # adjust the length as needed

# Print the unique fields
for doc in result:
    fields = doc['uniqueFields']
    print(fields)

['Synonym_German', 'Regional_Part_Of', 'JHU_White-Matter_Tractography_Atlas', 'Preferred_name', 'Branch_Of', 'Comment', 'Projects_To', 'Tributary_Of', 'Has_Blood_Supply', 'SNOMED_ID', 'FMAID', 'Related_modality', 'Receives_Drainage_From', 'Misspelling_of_term', 'Has_Entrapment_Site', 'May_Cause', 'Acronym', 'Sends_Output_To', 'Origin_of', 'Constitutional_Part_Of', 'Bounded_by', 'Unsanctioned_Term', 'Part_Of', 'Attaches_to', 'Receives_Projection_From', 'AAL', 'Has_Innervation_Source', 'Talairach', 'Synonym', 'Receives_Input_From', 'Blood_Supply_of', 'Preferred_name_German', 'Anatomical_Site', 'Lymphatic_Drainage_Of', 'Surrounded_by', 'Preferred_Name_for_Obsolete', 'Has_Constitutional_Part', 'Posterior_to', 'Bounds', 'JHU_DTI-81', 'Continuous_With', 'Has_insertion', 'Drains_Into', 'Projects_From', 'Inferior_to', 'External_to', 'Source', 'May_Be_Caused_By', 'Has_Part', 'Replaced_by', 'Member_Of', 'UMLS_Term', 'Proximal_to', 'Definition', 'Has_Branch', 'Radlex_version_of_class_change', 'Re

In [19]:
## len of unique fields
print(len(fields))

74


In [33]:
## is in list

query_list = ['Anatomical_Site', 'Comment', 'Definition', 'May_Be_Caused_By', 'is_a', 'Source', 'Related_modality', 'Preferred_name', 'Preferred_name_German']


# is in
for query in query_list:
    print(query in fields)

True
True
True
True
False
True
True
True
True


In [25]:
## transform to sets and compare
query_set = set(fields).difference(set(query_list))

## print the difference
print(query_set)
print(len(query_set))

{'Member_Of', 'Origin_of', 'External_to', 'Drains_Into', 'Receives_Drainage_From', 'Has_Member', 'Branch_Of', 'Has_Branch', 'Has_Innervation_Source', 'Has_Regional_Part', 'Misspelling_of_term', 'Lymphatic_Drainage', 'FMAID', 'Bounded_by', 'Insertion_of', 'Surrounded_by', 'Created', 'Acronym', 'AAL', 'JHU_DTI-81', 'Projects_To', 'Posterior_to', 'Unsanctioned_Term', 'JHU_White-Matter_Tractography_Atlas', 'Distal_to', 'Attaches_to', 'Has_Blood_Supply', 'Lymphatic_Drainage_Of', 'Receives_Input_From', 'Anterior_to', 'Synonym_German', 'Has_Part', 'Surrounds', 'Inferior_to', 'Segment_Of', 'Preferred_Name_for_Obsolete', 'Synonym', 'UMLS_Term', 'Regional_Part_Of', 'Contained_In', 'Proximal_to', 'Part_Of', 'Constitutional_Part_Of', 'Innervates', 'UMLS_ID', 'Talairach', 'Projects_From', 'Sends_Output_To', 'Has_origin', 'Has_Entrapment_Site', 'Continuous_With', 'Has_Constitutional_Part', 'Freesurfer', 'Has_insertion', 'Radlex_version_of_class_change', 'Blood_Supply_of', 'Tributary_Of', 'Bounds', '

In [32]:
# set to sorted list
query_list = sorted(list(query_set))

# print the sorted list
print(query_list)
print(len(query_list))

['AAL', 'Acronym', 'Anterior_to', 'Attaches_to', 'Blood_Supply_of', 'Bounded_by', 'Bounds', 'Branch_Of', 'CMA_Label', 'Constitutional_Part_Of', 'Contained_In', 'Contains', 'Continuous_With', 'Created', 'Distal_to', 'Drains_Into', 'External_to', 'FMAID', 'Freesurfer', 'Has_Blood_Supply', 'Has_Branch', 'Has_Constitutional_Part', 'Has_Entrapment_Site', 'Has_Innervation_Source', 'Has_Member', 'Has_Part', 'Has_Regional_Part', 'Has_insertion', 'Has_origin', 'Inferior_to', 'Innervates', 'Insertion_of', 'JHU_DTI-81', 'JHU_White-Matter_Tractography_Atlas', 'Lymphatic_Drainage', 'Lymphatic_Drainage_Of', 'May_Cause', 'Member_Of', 'Misspelling_of_term', 'Origin_of', 'Part_Of', 'Posterior_to', 'Preferred_Name_for_Obsolete', 'Projects_From', 'Projects_To', 'Proximal_to', 'Radlex_version_of_class_change', 'Receives_Drainage_From', 'Receives_Input_From', 'Receives_Projection_From', 'Receives_attachment_from', 'Regional_Part_Of', 'Replaced_by', 'SNOMED_ID', 'Segment_Of', 'Sends_Output_To', 'Superior_to

### Code to try transformation before writing to MongoDB

In [10]:
t = Transform(doc)

t_doc = t.transform_func()

print(t_doc.model_dump_json(indent=2))



{
  "id": "RID35591",
  "preferredLabel": "string-of-pearls sign of bowel",
  "synonyms": null,
  "parent": "RID29023",
  "definition": "Oblique or horizontal row of air bubbles visible on abdominal radiograph; almost always indicates small bowel obstruction; air is trapped between valvulae conniventes along the superior wall of the intestine.",
  "radlexProperties": {
    "anatomicalSite": "RID132",
    "comment": "157",
    "definition": "Oblique or horizontal row of air bubbles visible on abdominal radiograph; almost always indicates small bowel obstruction; air is trapped between valvulae conniventes along the superior wall of the intestine.",
    "mayBeCausedBy": "RID4962",
    "is_A": null,
    "source": "Radiology 2000; 214:157-158",
    "relatedModality": "RID10345",
    "preferredName": "string-of-pearls sign of bowel",
    "preferredNameGerman": "string-of-pearls sign of bowel (EN)"
  }
}


In [6]:
# Check multiple documents
docs = await collection.find({}).to_list(length=10)
for doc in docs:
    t = Transform(doc)
    t_doc = t.transform_func()
    print(t_doc.model_dump_json(indent=2))

{
  "id": "RID35591",
  "preferredLabel": "string-of-pearls sign of bowel",
  "synonyms": null,
  "parent": "RID29023",
  "definition": "Oblique or horizontal row of air bubbles visible on abdominal radiograph; almost always indicates small bowel obstruction; air is trapped between valvulae conniventes along the superior wall of the intestine.",
  "radlexProperties": {
    "anatomicalSite": "RID132",
    "comment": "157",
    "definition": "Oblique or horizontal row of air bubbles visible on abdominal radiograph; almost always indicates small bowel obstruction; air is trapped between valvulae conniventes along the superior wall of the intestine.",
    "mayBeCausedBy": "RID4962",
    "is_A": null,
    "source": "Radiology 2000; 214:157-158",
    "relatedModality": "RID10345",
    "preferredName": "string-of-pearls sign of bowel",
    "preferredNameGerman": "string-of-pearls sign of bowel (EN)"
  }
}
{
  "id": "RID45335",
  "preferredLabel": "peripheral segment of nerve to left sternocle

# Code to write transformed RadLex to new 'radlex' collection in MongoDB

In [4]:
# Create a list to store the transformed documents
transformed_docs = []


# iterate over documents in "RadLex" collection
for doc in radlex_old.find():
    # Transform the document
    transformed_doc = Transform(doc)
    # Append the transformed document to the list
    transformed_docs.append(transformed_doc)

NameError: name 'radlex_old' is not defined

In [ ]:
# Access the new "radlex" collection in MongoDB
radlex_new = db['radlex']



# write the transformed documents to the new "radlex" collection
radlex_new.insert_many(transformed_docs)